# Notebook for data split and transform for time series model 
## This notebook consists of steps to 
1. process raw data into independent time series for time series model 
2. train/test data split 

In [ ]:
import pandas as pd
import numpy as np

# Load raw data

In [2]:
raw_data_path = '../../data/01_raw/financial_fraud/bs140513_032310.csv'

In [3]:
raw_trans_data = pd.read_csv(raw_data_path)

In [4]:
raw_trans_data.shape

(594643, 10)

# Save (customer_id, merchant_id) that had more than 10 entries into time series 

In [291]:
raw_trans_data.shape

(594643, 10)

In [293]:
customer_merchant_trans_count = raw_trans_data.groupby(by=['customer','merchant']).agg({'step':'count'})
customer_merchant_trans_count

step
customer      merchant           
'C1000148617' 'M1053599405'     1
              'M1400236507'     1
              'M1741626453'     1
              'M1823072687'    92
              'M1842530320'     1
...                           ...
'C999723254'  'M348934600'     27
              'M349281107'      1
              'M480139044'      1
              'M855959430'      2
              'M85975013'       8

[47132 rows x 1 columns]

In [294]:
ts_count_threshold = 10

In [297]:
customer_merchant_trans_more_than_threshold =customer_merchant_trans_count.loc[customer_merchant_trans_count.step>ts_count_threshold].reset_index()

In [298]:
customer_merchant_trans_more_than_threshold

,customer,merchant,step
0,'C1000148617','M1823072687',92
1,'C1000148617','M348934600',22
2,'C100045114','M1823072687',27
3,'C100045114','M348934600',38
4,'C1000699316','M1823072687',52
...,...,...,...
7853,'C998987490','M85975013',13
7854,'C999393223','M1823072687',19
7855,'C999393223','M348934600',90
7856,'C999723254','M1823072687',77


In [299]:
customer_merchant_pairs_for_ts = np.array(customer_merchant_trans_more_than_threshold[['customer','merchant']])

In [301]:
customer_merchant_pairs_for_ts, customer_merchant_pairs_for_ts.shape

(array([["'C1000148617'", "'M1823072687'"],
        ["'C1000148617'", "'M348934600'"],
        ["'C100045114'", "'M1823072687'"],
        ...,
        ["'C999393223'", "'M348934600'"],
        ["'C999723254'", "'M1823072687'"],
        ["'C999723254'", "'M348934600'"]], dtype=object),
 (7858, 2))